# Step 11, 가변 길이 인수(순전파 편)

지금까지 우리는 함수에 입출력 변수가 하나씩인 경우만 생각해왔다.  
그러나 함수에 따라 여러개의 변수를 입력받기도 한다.(예:덧셈,곱셉등)  
또한 여려개의 변수를 출력하기도 한다.

이것을 고려하여 DeZero가 가변길이 입축력을 처리할 수 있도록 확장하려고 한다.

## 11.1 Function 클래스 수정

가변길이 입출력을 표현하려면 변수들을 리스트(또는 튜플)에 넣어 처리  
즉, Function 클래스는 지금까지처럼 '하나의 인수'를 받고 '하나의 값'만 반환하지만  
인수와 반환값의 타입을 리스트로 바꾸고, 필요한 변수들을 이 리스트에 넣으면 된다.

In [4]:
from DeZero_test import Variable, as_array
import numpy as np

In [5]:
# 기존 Function 클래스 구조
class Function:
    def __init__(self,input):
        x = input.data                  # 1
        y = self.forward(x)             # 2
        output = Variable(as_array(y))  # 3
        output.set_creator(self)        # 4
        self.input = input
        self.output = output
        return output
    
    def forward(self,x):
        raise NotImplementedError() 

    def backward(self,gy):
        raise NotImplementedError()

- 1 Function의 __call__ 메서드는 Variable 이라는 '상자'에서 실제 데이터를 꺼낸다.  
- 2 forward 메서드에서 구체적인 계산 
- 3 계산 결과를 Variable에 넣고 
- 4 자신이 '창조자'라고 원산지 표시를 한다.

다음 로직을 염두에 두고 __call__ 메서드의 인수와 반환값을 리스트로 바꿔본다.

In [6]:
class Function:
    def __call__(self,inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)

        self.inputs = inputs
        self.outputs = outputs
        return outputs
    
    def forward(self,xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

인수와 반환값을 리스트로 변경  
변수를 리스트에 담아 취급한다는 점을 제외하고 달라진게 없음

## 11.2 Add 클래스 구현

이번 절에서는 Add 클래스의 forward 메서드를 구현.
주의할 점은 인수와 반환값이 리스트여야 한다

In [7]:
class Add(Function):
    def forward(self,xs):
        x0, x1 = xs
        y = x0 + x1 
        return (y,)

Add 클래스의 인수는 변수가 두 개 담긴 리스트  
따라서 xs에서 원소 2개를 꺼냈다.  
그런 다음 꺼낸 원소들을 사용하여 계산  
결과를 반환할때는 return (y,) 형태로 튜플을 반환 

In [8]:
xs = [Variable(np.array(2)), Variable(np.array(3))]     # 리스트로 준비
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


이제 순전파에 한해서는 가변 길이 인수와 반환값에 대응할 수 있다.  
그런데 앞의 코드는 다소 귀찮게 구성되어있다. 입력변수를 리스트에 담아 건네주어야하고 반환값은 튜플로 받게 한다.

# Step12, 가변 길이 인수(개선 편)

두가지 개선  
1. Add 클래스(혹은 다른 구체적인 함수 클래스)를 '사용하는 사람'을 위한 개선  
2. '구현하는 사람'을 위한 개선

## 12.1 첫번째 개선: 함수를 사용하기 쉽게

현재 코드  
~~~python
xs = [Variable(np.array(2)),
      Variable(np.array(3))]

f = Add()
ys = f(xs)
y = ys[0]
~~~

이런 코드를 아래와 같이 개선

~~~python 
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))

f = Add()

y = f(x0,x1)
~~~

In [9]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

Function 클래스 수정

In [10]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [11]:
class Add(Function):
    def forward(self,xs):
        x0, x1 = xs
        y = x0 + x1 
        return (y,)

2. outputs에 원소가 하나뿐이라면 리스트가 아니라 그 원소만을 반환. 즉 함수의 반환값이 하나라면 해당 변수를 직접 리턴 
1. 함수를 정의 할때 인수 앞에 *를 붙인다. 리스트를 사용하는 대신 임의 개수의 인수(가변 길이 인수)를 건네 함수를 호출할 수 있다.  

함수를 정의할때 인수에 별표를 붙이면 호출할 때 넘긴 인수들을 별표를 붙인 인수 하나로 모아서 받을 수 있다.

가변 길이 인수 사용법

In [12]:
# 가변 길이 인수 사용법 
def f(*x):
    print(x)

print(f(1,2,3))

print(f(1,2,3,4,5))

(1, 2, 3)
None
(1, 2, 3, 4, 5)
None


In [13]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))

f = Add()
y = f(x0,x1)
print(y.data)

5


## 12.2 두번째 개선: 함수를 구현하기 쉽도록 

Add 클래스를 '구현하는 사람'을 위한 개선 

현재 코드

~~~python
class Add(Function):
    def forward(self,xs):
        x0, x1 = xs 
        y = x0 + x1 
        return (y,)
~~~

개선 코드 

~~~python 
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1 
        return y
~~~


두번째 개선을 위해 Function 클래스를 수정


In [14]:
class Function:
    def __call__(self,*inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)                              # 1. *를 붙여 언팩
        if not isinstance(ys,tuple):                        # 2. 튜플이 아닌 경우 추가 지원
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

1. self.forward(*xs)  
 - 함수를 "호출"할 때 별표를 붙였는데, 이렇게 하면 리스트 언팩(list unpack)이 이루어짐
 - 언팩은 리스트의 원소를 낱개로 풀어서 전달하는 기법 
    - 예 : xs = [x0,x1]일때 self.forward(*xs)를 하면 self.forward(x0,x1)로 호출하는 것과 동일하게 동작

2. ys가 튜플이 아닌경우 튜플로 변경 

In [15]:
class Add(Function):
    def forward(self,x0,x1):
        y = x0 + x1 
        return y

## 12.3 add 함수 구현 

In [16]:
def add(x0,x1):
    return Add()(x0,x1)

최종 코드

In [17]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y


def add(x0, x1):
    return Add()(x0, x1)


x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5
